In [7]:
from collections import Counter
import os
import pickle
import glob
import gspread


import matplotlib.ticker as tkr
from IPython.core.display import HTML
from oauth2client.client import SignedJwtAssertionCredentials
import json
import pandas as pd
import scipy.spatial
import scipy.cluster
import seaborn as sns

from gscripts.general import parsers
from gscripts.general.dataviz import Figure
from gscripts.GO import GO
import gffutils
from scipy.stats import hypergeom




img_path = "/nas3/gpratt/Dropbox/Presentations/notebooks/public_clip"

In [10]:
json_key = json.load(open("/home/gpratt/ipython_notebook/public clip-588adbc137f3.json"))
scope = ['https://spreadsheets.google.com/feeds']

credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)
gc = gspread.authorize(credentials)

sht1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1ZU2mQh54jentqvhR_oMnviLGWR8Nw_x338gULzKjNDI/edit#gid=0")
ws = sht1.worksheet("public_clip_database")
list_of_lists = ws.get_all_values()
df = pd.DataFrame(list_of_lists[1:], columns=list_of_lists[0])
df.index = df.SRA

public_clip_list = df


In [13]:
clip_rnaseq_df = parsers.clipseq_metrics("/home/gpratt/projects/public_clip/analysis/public_clip_v10/", iclip=False)
clip_rnaseq_df = pd.merge(clip_rnaseq_df, public_clip_list, left_index=True, right_index=True, how="outer")
clip_df = clip_rnaseq_df[clip_rnaseq_df['CLIP-seq?'] == "Yes"]


/home/gpratt/projects/public_clip/analysis/public_clip_v10/SRR363798.polyATrim.adapterTrim.metrics

/home/gpratt/projects/public_clip/analysis/public_clip_v10/SRR363799.polyATrim.adapterTrim.metrics

/home/gpratt/projects/public_clip/analysis/public_clip_v10/SRR850555.polyATrim.adapterTrim.metrics

/home/gpratt/projects/public_clip/analysis/public_clip_v10/SRR850554.polyATrim.adapterTrim.metrics

/home/gpratt/projects/public_clip/analysis/public_clip_v10/SRR850556.polyATrim.adapterTrim.metrics

/home/gpratt/projects/public_clip/analysis/public_clip_v10/SRR850551.polyATrim.adapterTrim.metrics

/home/gpratt/projects/public_clip/analysis/public_clip_v10/SRR363796.polyATrim.adapterTrim.metrics

/home/gpratt/projects/public_clip/analysis/public_clip_v10/SRR363797.polyATrim.adapterTrim.metrics

/home/gpratt/projects/public_clip/analysis/public_clip_v10/SRR867250.polyATrim.adapterTrim.metrics

/home/gpratt/projects/public_clip/analysis/public_clip_v10/SRR850552.polyATrim.adapterTrim.metrics


In [15]:
filtered_df = clip_df[[ "Input Reads",
                        #"Reads After Triming",
                        "Reads Passing Quality Filter", 
                        "Uniquely Mapped Reads",
                        #"Uniquely mapped reads %",
                        "Usable Reads",
                            #"spot",
                            "Num Peaks",
                            #"%usable / input",
                            #"%usable / mapped"
                            ]]


HTML(filtered_df.to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads After Triming" : parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Uniquely mapped reads number" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))

,Input Reads,Reads Passing Quality Filter,Uniquely Mapped Reads,Usable Reads,Num Peaks
ERR018282,0,NaN,NaN,0,0
ERR018283,0,NaN,NaN,0,0
ERR018284,0,NaN,NaN,0,0
ERR039833,0,NaN,NaN,0,0
ERR039834,0,NaN,NaN,0,0
ERR039835,0,NaN,NaN,0,0
ERR039836,0,NaN,NaN,0,0
ERR039837,0,NaN,NaN,0,0
ERR039838,0,NaN,NaN,0,0
ERR039839,0,NaN,NaN,0,0
